# [실습] LangChain Tool Call과 Agent

LangChain의 기본 코드를 통해, Tool Call과 Agent가 작동하는 과정에 대해 알아보겠습니다.   

기본 라이브러리를 설치합니다.

In [ ]:
!pip install -U langchain_google_genai langchain_core langchain_community tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninst

Google API Key를 설정합니다.

구글 로그인 후, https://aistudio.google.com/apikey 에서 발급받을 수 있습니다.

In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = ''

Gemini 모델을 설정합니다.   
무료 API의 경우 분당 제한이 존재하므로, 랭체인의 Rate Limiter를 적용할 수 있습니다.

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    rate_limiter=rate_limiter
)

<ipython-input-3-1f5f89156181>:6: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


`invoke()`를 통해 llm에 입력을 전달합니다.    
랭체인 기본 클래스인 `Message` 계열 클래스를 직접 만들어 리스트로 전달하거나,   
프롬프트 템플릿을 통해 입력의 구성을 만들어 전달할 수 있습니다.

In [ ]:
# 1. HumanMessage를 이용한 방법

from langchain_core.messages import HumanMessage, SystemMessage

system_msg = SystemMessage(content="""사용자의 아래 [질문]에 대해 친절하게 답변하세요.
답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.""")
msg = HumanMessage(content="안녕! 너는 이름이 뭐니?")

messages = [system_msg, msg]

response = llm.invoke(messages)
response

AIMessage(content='안녕하세요! 저는 구글에서 개발한 인공지능 모델이고, 특별한 이름은 없습니다. \n\n*   저는 사용자의 질문에 답변하고, 정보를 제공하며, 다양한 작업을 수행할 수 있습니다.\n*   저는 끊임없이 학습하고 발전하고 있습니다.\n*   궁금한 점이 있다면 언제든지 저에게 물어보세요.\n*   저는 사용자를 돕기 위해 존재합니다.\n*   앞으로 저와 함께 즐거운 시간을 보내시길 바랍니다!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e8f1fd1c-35e2-4cd3-9c1c-ec4b4144d818-0', usage_metadata={'input_tokens': 57, 'output_tokens': 135, 'total_tokens': 192, 'input_token_details': {'cache_read': 0}})

Chat 모델의 출력은 AIMessage라는 형식으로 변환됩니다.

In [ ]:
print(response.content)

안녕하세요! 저는 구글에서 개발한 인공지능 모델이고, 특별한 이름은 없습니다. 

*   저는 사용자의 질문에 답변하고, 정보를 제공하며, 다양한 작업을 수행할 수 있습니다.
*   저는 끊임없이 학습하고 발전하고 있습니다.
*   궁금한 점이 있다면 언제든지 저에게 물어보세요.
*   저는 사용자를 돕기 위해 존재합니다.
*   앞으로 저와 함께 즐거운 시간을 보내시길 바랍니다!


In [ ]:
# 2. ChatPromptTemplate을 이용한 방법

from langchain.prompts import ChatPromptTemplate

# 입력 변수가 {question}인 템플릿
prompt = ChatPromptTemplate([
    ('system', '''사용자의 아래 [질문]에 대해 친절하게 답변하세요.
     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'''),
     ('user', '''[질문]: {question}''')])
prompt


ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='사용자의 아래 [질문]에 대해 친절하게 답변하세요. \n     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='[질문]: {question}'), additional_kwargs={})])

프롬프트와 llm을 |로 연결하여 체인을 구성합니다.   
(랭체인 기본 문법으로, LangChain 기초 복습 자료에서 자세히 보실 수 있습니다!)

In [ ]:
chain = prompt | llm
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='사용자의 아래 [질문]에 대해 친절하게 답변하세요. \n     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='[질문]: {question}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x7992290177d0>, model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x799228f51890>, default_metadata=())

In [ ]:
response = chain.invoke({'question':"랭체인과 랭그래프의 차이가 뭐야?"})

print(response.content)

랭체인(LangChain)과 랭그래프(LangGraph)는 모두 LLM(Large Language Model)을 활용한 애플리케이션 개발을 돕는 프레임워크이지만, 주요 차이점은 다음과 같습니다:

*   **랭체인**: LLM을 연결하여 복잡한 작업을 수행하는 데 초점을 맞춘 모듈형 프레임워크입니다.
*   **랭그래프**: 멀티 에이전트 시스템을 구축하기 위한 프레임워크로, 에이전트 간의 상호작용 및 워크플로우 관리에 특화되어 있습니다.
*   **랭체인**: 다양한 LLM, 데이터 소스, 유틸리티를 연결하는 데 용이합니다.
*   **랭그래프**: 에이전트들이 상태를 공유하고, 결정을 내리고, 작업을 반복/병렬적으로 수행하는 데 최적화되어 있습니다.
*   **요약**: 랭체인은 LLM 연결 및 활용에, 랭그래프는 멀티 에이전트 시스템 구축 및 관리에 더 적합합니다.


In [ ]:
# 입력 변수가 1개일 때는 값만 invoke해도 됨

response = chain.invoke("패스트캠퍼스 랭그래프 강의 이름이 뭐야?")

print(response.content)

패스트캠퍼스에서 제공하는 랭그래프 강의는 다음과 같습니다.

*   **[풀스택] 랭체인 기반 AI 에이전트 개발**
*   **[자동화] 랭체인으로 시작하는 LLM Ops**

두 강의 모두 랭체인을 활용하여 인공지능 에이전트 개발 및 LLM Ops 자동화를 다루고 있습니다. 풀스택 강의는 에이전트 개발에, 자동화 강의는 LLM Ops에 더 집중하는 차이가 있습니다.


존재하지 않는 내용에 대한 환각이 발생한 모습입니다.

## LLM Tool 설정하기

Tool은 LLM이 사용할 수 있는 다양한 외부 기능을 의미합니다.   
LangChain에서 자체적으로 지원하는 Tool을 사용하거나, RAG의 Retriever를 Tool로 변환하는 것도 가능합니다.     
또한, 함수를 Tool로 변환할 수도 있습니다.

1. Tavily Search (http://app.tavily.com/)

Tavily는 AI 기반의 검색 엔진입니다. 계정별 월 1000개의 무료 사용량을 지원합니다.      
Tavily Search는 URL과 함께 내용의 간단한 요약을 지원하는 것이 특징입니다.


In [ ]:
os.environ['TAVILY_API_KEY'] = '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'


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    )
tavily_search

TavilySearchResults(include_answer=True, include_raw_content=True, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))

In [ ]:
search_docs = tavily_search.invoke("랭그래프와 랭체인의 차이")
search_docs

[{'url': 'https://wikidocs.net/268613',
  'content': 'Langchain 랭체인과 Langgraph의 차이점 - 모두의 AI 케인의 LangGraph로 끝내는 멀티 AI Agent : LangGraph 시작하기 랭 체인은 여러 구성 요소를 선형적으로 이어주는 구조로, 정의된 순서에 따라 작업을 수행하는 반면, 랭 그래프는 노드와 엣지의 연결을 통해 비선형적이고 동적인 프로세스를 구성합니다. 체인 홈페이지에서 랭 체인과 랭 그래프가 평행선상에 표현되어 있는 그림을 볼 수 있다 따라서, 랭 체인 생태 내에서 두 프레임워크는 주요 아키텍처로서 비슷한 점이 많다고 볼 수 있다 랭 체인의 주요 활용은 레그라로 볼 수 있다 레그의 인덱싱 과정은 일종의 파이프라인을 통해 수행되며, 랭 체인을 활용하여 생성 과정을 완성할 수 있다 랭 체인은 일방향적 체인 구조로 구성할 수 있지만, 랭 그래프는 주어진 메시지에 따라 어시스턴트가 다음 노드를 선택한다 Report a comment. Do you want to report this comment?'},
 {'url': 'https://wikidocs.net/261585',
  'content': 'LangGraph와 LangChain의 차이점 - LangGraph 가이드북 - 에이전트 RAG with 랭그래프 | 주요 목적 | 복잡한 워크플로우 및 의사결정 프로세스 구현 | LLM 통합 및 체인 구성 | | 용도 | 복잡한 AI 시스템, 다중 에이전트 | 간단한 LLM 애플리케이션, RAG | 개발자가 각 단계에서 상태를 직접 제어하고 수정할 수 있어, 복잡한 상태 변화를 정확하게 추적하고 관리할 수 있습니다. LangChain은 미리 정의된 컴포넌트를 중심으로 구성되어 있어, 기본적인 기능을 빠르게 구현할 수 있지만 고도로 커스터마이즈된 로직을 구현하는 데는 상대적으로 제한이 있을 수 있습니다. LangGraph는 복잡한 AI 시스템이나 다중 에이전트 시스템을 구축하는 데 적합합니다. 

2.

# LLM에 Tool 연결하기   

생성한 툴은 llm.bind_tools()를 통해 LLM에 연결할 수 있습니다.    
이 기능은 Gemini, GPT, Claude 등의 툴에서 지원하는데요.     
HuggingFace 공개 모델에서 Tool을 사용하는 경우는 이후 실습에서 자세히 알아보겠습니다!

In [ ]:
tools = [tavily_search]

llm_with_tools = llm.bind_tools(tools)

llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x7992290177d0>, model='models/gemini-2.0-flash-exp', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x799228f51890>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

랭체인에서, tool 정보는 tools에 저장되는데요.   
해당 내용은 랭체인 내부에서 json Schema 형식으로 프롬프트에 포함됩니다.

In [ ]:
# 시스템 메시지에 포함되는 내용
# (`LLM과 Agent` 강의 슬라이드를 참고하세요!)

llm_with_tools.kwargs['tools']

[{'type': 'function',
  'function': {'name': 'tavily_search_results_json',
   'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
   'parameters': {'properties': {'query': {'description': 'search query to look up',
      'type': 'string'}},
    'required': ['query'],
    'type': 'object'}}}]

LLM은 프롬프트로 주어지는 툴 정보를 바탕으로 Tool의 사용을 결정합니다.    
Schema를 통해, 툴의 의미와 사용 방법, 형식을 이해합니다.

이후, LLM은 Tool 사용이 필요한 경우 특별한 포맷의 메시지를 출력합니다.   
이를 Tool Call Message라고 부릅니다.

In [ ]:
tool_chain = prompt | llm_with_tools
tool_call_msg = tool_chain.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")
tool_call_msg


AIMessage(content='현재 사용할 수 있는 API에는 패스트캠퍼스에 대한 정보가 없습니다. 따라서 해당 과정이 어떤 과정인지에 대한 질문에 답변을 드릴 수 없습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d39f412c-08eb-40a9-9d1a-7e8a3104f681-0', usage_metadata={'input_tokens': 133, 'output_tokens': 46, 'total_tokens': 179, 'input_token_details': {'cache_read': 0}})

In [ ]:
tool_call_msg = llm_with_tools.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정인지 검색해서 알려줘")
tool_call_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "\\ud328\\uc2a4\\ud2b8\\ucea0\\ud37c\\uc2a4 \\ub7ad\\uadf8\\ub798\\ud504\\ub85c \\ud55c\\ubc88\\uc5d0 \\uc644\\uc131\\ud558\\ub294 \\ubcf5\\uc7a1\\ud55c RAG\\uc640 Agent \\uacfc\\uc815"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-564cafb1-07aa-497a-ba3a-5574581a2535-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'}, 'id': 'ecb03e78-5aa2-40f9-95bf-f3cfde028a38', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 34, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}})

In [ ]:
tool_call_msg.tool_calls
# 리스트 형식인 이유는? 여러 개의 툴을 동시에 사용할 수 있기 때문문

[{'name': 'tavily_search_results_json',
  'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'},
  'id': 'ecb03e78-5aa2-40f9-95bf-f3cfde028a38',
  'type': 'tool_call'}]

`tool_calls`에 포함된 name을 활용하면 툴 결과를 전달할 수 있습니다.   
`name` 값은 문자열이므로, dictionary를 통해 연결합니다.

In [ ]:
tool_list ={'tavily_search_results_json': tavily_search}

tool_name = tool_call_msg.tool_calls[0]['name']
tool_exec = tool_list[tool_name]

tool_name, tool_exec

('tavily_search_results_json',
 TavilySearchResults(include_answer=True, include_raw_content=True, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))))

Tool에 tool_call을 입력해 invoke를 수행합니다.

In [ ]:
tool_msg = tool_exec.invoke(tool_call_msg.tool_calls[0])
tool_msg

ToolMessage(content='[{"url": "https://fastcampus.co.kr/new", "content": "서강대학교 DX-AI miniMBA 과정. 15주 과정약 45시간직무경험 필요 ... 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent. 평생소장약 20시간사전지식 필요."}, {"url": "https://fastcampus.co.kr/data_online_langgraph", "content": "랭그래프로 한번에 완성하는 복잡한 RAG와 Agent | 패스트캠퍼스 #LLM#multi agent#RAG#랭그래프 결제 수단에 따라 무이자 할부가 불가할 수 있습니다. 복잡한 RAG와 Agent 설계를 완성하는 LangGraph 첫 걸음 5가지 Advanced AI agent 구축 프로젝트 약 16시간사전지식 필요 FC AWARD 본 패키지는 약 20시간 분량으로, 일 1시간 내외의 학습 시간을 통해 정상 수강 기간(=유료 수강 기간) 내에 모두 수강이 가능합니다. 수강시작 후 7일 이내, 5강 미만 수강 시에는 100% 환불 가능합니다. 수강시작 후 7일 이내, 5강 이상 수강 시 전체 강의에서 수강한 강의의 비율에 해당하는 수강료를 차감 후 환불 가능합니다. 수강시작 후 7일 초과 시 정상 수강기간 대비 잔여일에 대해 아래 환불규정에 따라 환불 가능합니다. 패스트캠퍼스 온라인 강의 시청을 위해서는 ID별 최대 4개의 기기를 등록할 수 있으며, 기기 등록은 온라인 강의장 접속 시 자동 등록됩니다."}, {"url": "https://kr.linkedin.com/posts/hyungho-byun-6b2588224_%EB%9E%AD%EA%B7%B8%EB%9E%98%ED%94%84%EB%A1%9C-%ED%95%9C%EB%B2%88%EC%97%90-%EC%99%84%EC%84%B1%ED%95%98%EB%8A%94-%EB%B3%B5%EC%9E%A1%ED%95%9C-rag%EC%99%80-agent-%ED%8C%A8%EC%8A%A4%ED

이번에는 ToolMessage라는 새로운 형태의 메시지가 생성되었습니다!

툴 실행 결과를 LLM에게 다시 전달하여, 이를 해석하고 답변하게 만들어 보겠습니다.   

Chain 형식의 구성은 메시지를 변화시키는 것이 복잡하므로, 이전에 사용했던 `HumanMessage`에서 시작하여   
LLM의 답변인 `AIMessage`와 `ToolMessage` 를 추가합니다.

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")]

messages.append(tool_call_msg)
messages.append(tool_msg)

messages

[HumanMessage(content="패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "\\ud328\\uc2a4\\ud2b8\\ucea0\\ud37c\\uc2a4 \\ub7ad\\uadf8\\ub798\\ud504\\ub85c \\ud55c\\ubc88\\uc5d0 \\uc644\\uc131\\ud558\\ub294 \\ubcf5\\uc7a1\\ud55c RAG\\uc640 Agent \\uacfc\\uc815"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-564cafb1-07aa-497a-ba3a-5574581a2535-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'}, 'id': 'ecb03e78-5aa2-40f9-95bf-f3cfde028a38', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 34, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"url": "https://fastcampus.co.kr/new", "content": "서강대학교 D

In [ ]:
print(messages[2].content)

[{"url": "https://fastcampus.co.kr/new", "content": "서강대학교 DX-AI miniMBA 과정. 15주 과정약 45시간직무경험 필요 ... 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent. 평생소장약 20시간사전지식 필요."}, {"url": "https://fastcampus.co.kr/data_online_langgraph", "content": "랭그래프로 한번에 완성하는 복잡한 RAG와 Agent | 패스트캠퍼스 #LLM#multi agent#RAG#랭그래프 결제 수단에 따라 무이자 할부가 불가할 수 있습니다. 복잡한 RAG와 Agent 설계를 완성하는 LangGraph 첫 걸음 5가지 Advanced AI agent 구축 프로젝트 약 16시간사전지식 필요 FC AWARD 본 패키지는 약 20시간 분량으로, 일 1시간 내외의 학습 시간을 통해 정상 수강 기간(=유료 수강 기간) 내에 모두 수강이 가능합니다. 수강시작 후 7일 이내, 5강 미만 수강 시에는 100% 환불 가능합니다. 수강시작 후 7일 이내, 5강 이상 수강 시 전체 강의에서 수강한 강의의 비율에 해당하는 수강료를 차감 후 환불 가능합니다. 수강시작 후 7일 초과 시 정상 수강기간 대비 잔여일에 대해 아래 환불규정에 따라 환불 가능합니다. 패스트캠퍼스 온라인 강의 시청을 위해서는 ID별 최대 4개의 기기를 등록할 수 있으며, 기기 등록은 온라인 강의장 접속 시 자동 등록됩니다."}, {"url": "https://kr.linkedin.com/posts/hyungho-byun-6b2588224_%EB%9E%AD%EA%B7%B8%EB%9E%98%ED%94%84%EB%A1%9C-%ED%95%9C%EB%B2%88%EC%97%90-%EC%99%84%EC%84%B1%ED%95%98%EB%8A%94-%EB%B3%B5%EC%9E%A1%ED%95%9C-rag%EC%99%80-agent-%ED%8C%A8%EC%8A%A4%ED%8A%B8%EC%BA%A0%ED%8D

`Query-Tool Call-Tool`의 형식은 가장 기본적인 툴 사용 방법입니다.

In [ ]:
result = llm_with_tools.invoke(messages)
# 검색 결과를 바탕으로 답변한 모습

print(result.content)


패스트캠퍼스의 "랭그래프로 한번에 완성하는 복잡한 RAG와 Agent" 과정은 다음과 같은 특징을 가진 것으로 보입니다:

*   **주요 내용:** LangGraph 라이브러리를 사용하여 복잡한 RAG (Retrieval-Augmented Generation) 및 Agent 설계를 완성하는 방법을 다룹니다. LangGraph는 LLM 어플리케이션 개발에서 복잡한 워크플로우를 구성하고 관리하는 데 유용한 라이브러리입니다.
*   **학습 목표:** Advanced AI agent 구축 프로젝트를 통해 RAG와 Agent 설계를 익히는 것을 목표로 합니다.
*   **수강 시간:** 약 20시간 분량으로 구성되어 있습니다.
*   **사전 지식:** 사전 지식이 필요한 과정입니다.
*   **환불 정책:**
    *   수강 시작 후 7일 이내, 5강 미만 수강 시 100% 환불 가능합니다.
    *   수강 시작 후 7일 이내, 5강 이상 수강 시 전체 강의에서 수강한 강의의 비율에 해당하는 수강료를 차감 후 환불 가능합니다.
    *   수강 시작 후 7일 초과 시 정상 수강기간 대비 잔여일에 대해 환불 규정에 따라 환불 가능합니다.


이제, 지금까지 배운 내용을 종합하여 실행해 보겠습니다.   
질문이 주어지면, 질문에 대한 검색을 수행하여 결과를 바탕으로 답변합니다.   

<br><br>

그런데, 이번에는 분기가 필요합니다.   
검색이 필요없는 질문이라면, LLM은 tool을 요청하지 않을 것입니다.

In [ ]:
# LLM, Question, Tool을 이용한 간단한 함수
def simple_workflow(llm, question , tools = [tavily_search]):

    # 툴과 LLM 구성

    tool_list = {x.name: x for x in tools}
    # tavily_search.name = 'tavily_search_results_json' 을 이용하면
    # tool_list = {'tavily_search_results_json': tavily_search} 와 동일합니다.

    llm_with_tools = llm.bind_tools(tools)


    # 메시지 구성
    messages = [HumanMessage(content=question)]
    print('Query:', question)


    # LLM에 메시지 전달 (분기)
    tool_msg = llm_with_tools.invoke(question)
    messages.append(tool_msg)

    if tool_msg.tool_calls:
        # 툴 호출이 있을 경우: 툴 실행 후 결과를 전달
        tool_name = tool_msg.tool_calls[0]['name']

        print(f"-- {tool_name} 사용 중 --")
        print(tool_msg.tool_calls[0])


        tool_exec = tool_list[tool_name]

        tool_result = tool_exec.invoke(tool_msg.tool_calls[0])
        messages.append(tool_result)
        result = llm_with_tools.invoke(messages)

    else:
        # 툴 호출이 없을 경우: 처음 출력을 그대로 전달
        result = tool_msg

    return result.content

In [ ]:
response = simple_workflow(llm, "패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정인지 검색해줘.")
print(response)

Query: 패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정인지 검색해줘.
-- tavily_search_results_json 사용 중 --
{'name': 'tavily_search_results_json', 'args': {'query': '패스트캠퍼스 랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'}, 'id': '00dad3d8-66f9-4178-bc92-32f295bb4c58', 'type': 'tool_call'}
패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 다음과 같은 특징을 가지고 있습니다:

*   **주요 내용:** 복잡한 RAG (Retrieval-Augmented Generation) 및 Agent 설계를 LangGraph를 사용하여 완성하는 방법을 다룹니다. LangGraph는 LLM 어플리케이션 개발을 위한 라이브러리이며, 복잡한 워크플로우를 구성하고 관리하는 데 유용합니다.
*   **분량:** 약 20시간 분량이며, 일 1시간 내외의 학습으로 수강 기간 내에 완료할 수 있습니다.
*   **환불 규정:** 수강 시작 후 7일 이내, 5강 미만 수강 시 100% 환불 가능하며, 5강 이상 수강 시에는 수강한 강의 비율에 따라 환불 금액이 달라집니다. 7일 초과 시에는 잔여일에 따라 환불 규정이 적용됩니다.
*   **기기 등록:** ID별 최대 4개의 기기를 등록하여 온라인 강의를 시청할 수 있습니다.
*   **사전 지식:** 사전 지식이 필요합니다.

이 과정은 LLM (Large Language Model) 어플리케이션 개발에 관심 있는 사람들에게 유용할 것으로 보입니다. 특히, 랭체인(LangChain)을 사용하여 개발해 본 경험이 있다면 LangGraph를 통해 더욱 복잡한 워크플로우를 구축하고 관리하는 데 도움이 될 것입니다.


# Custom Tool 만들기

Tavily Search의 경우, 랭체인에서 사전에 구성한 기본 Schema가 존재하지만,   

Tool을 새로 만드는 경우에는 description과 같은 값을 직접 구성해야 합니다.   
가장 간단한 방식은 랭체인의 Tool 데코레이터를 사용하는 것입니다.  

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(x:int, y:int) -> int:
    "x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다."
    return x*y

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

print(multiply.invoke({'x':3, 'y':4}))
print(current_date.invoke({}))


12
2025-02-09


In [ ]:
tools = [tavily_search, multiply, current_date]

In [ ]:
question = '오늘 날짜는?'

result = simple_workflow(llm, question, tools)
result


Query: 오늘 날짜는?
-- current_date 사용 중 --
{'name': 'current_date', 'args': {}, 'id': '5a99c703-7a3e-4e79-ab7a-06985148b47e', 'type': 'tool_call'}


'오늘 날짜는 2025-02-09 입니다.'

In [ ]:
# LLM은 오늘 날짜를 모른다
llm.invoke('오늘 날짜는?').content

'오늘 날짜는 2024년 5월 15일입니다.'

<br><br>
## [연습문제] 복권 숫자 예측시키기

아래의 함수에 대해, 함수의 설명을 추가하여 llm의 툴로 전달해 봅시다.

In [ ]:
from typing import List
import random

@tool
def generate_random_numbers(min_val: int, max_val: int, count: int) -> List[int]:
    """주어진 범위 내에서 지정된 개수만큼의 중복되지 않은 랜덤 정수를 생성합니다.
    Args:
        min_val (int): 최솟값
        max_val (int): 최댓값
        count (int): 생성할 숫자의 개수

    Returns:
        List[int]: 중복되지 않은 랜덤 정수들의 리스트
    """
    # possible_range = max_val - min_val + 1
    # if count > possible_range:
    #     raise ValueError(f"생성 가능한 숫자의 범위({possible_range}개)보다 더 많은 숫자({count}개)를 요청했습니다.")

    return str(random.sample(range(min_val, max_val + 1), count))

In [ ]:
# LLM에 함수 Bind

llm_lotto = llm.bind_tools([generate_random_numbers])
llm_lotto.invoke('로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.')

AIMessage(content='', additional_kwargs={'function_call': {'name': 'generate_random_numbers', 'arguments': '{"min_val": 1.0, "count": 6.0, "max_val": 45.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-50feaf9d-5641-4b85-898d-ce7c57acaef8-0', tool_calls=[{'name': 'generate_random_numbers', 'args': {'min_val': 1.0, 'count': 6.0, 'max_val': 45.0}, 'id': '4de8b8b1-edd2-478f-aa2a-f79e926f6334', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 12, 'total_tokens': 149, 'input_token_details': {'cache_read': 0}})

In [ ]:
# simple_workflow 함수를 이용하여 실행하기

simple_workflow(llm_lotto, '로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.', [generate_random_numbers])

Query: 로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.
-- generate_random_numbers 사용 중 --
{'name': 'generate_random_numbers', 'args': {'min_val': 1.0, 'count': 6.0, 'max_val': 45.0}, 'id': 'b0d1ba38-0783-4b2d-8420-0780bf88ea66', 'type': 'tool_call'}


'로또 번호는 10, 27, 39, 30, 20, 14 입니다!\n```'

우리가 만든 `simple_workflow` 함수는 가장 간단한 형태의 Single Tool Calling을 수행하는 방식이었는데요.   

실제 환경의 시나리오는 훨씬 복잡합니다.   
한 번에 여러 개의 툴을 실행하거나, 툴 실행 결과를 받아 다음 툴에 활용할 수도 있을 것입니다.   

랭체인에서도 이와 같이 복잡한 작업을 수행하기 위한 기능이 자체적으로 구현되어 있는데요.    

예시로 `Structured Chat Agent`를 간단하게 만들고 실행해 보겠습니다.

# Agent

생각-행동-관찰을 거치는 ReAct 에이전트는 복잡한 플로우를 효과적으로 처리합니다.   
- 생각(Thought): 주어진 Context 상에서 다음 작업을 어떻게 수행할지 설명하는 과정
- 행동(Action): Tool 실행 명령어를 생성하는 과정
- 관찰(Thought): Tool 결과를 Context에 추가하는 과정


이 과정은 랭체인에 구성된 `crate_structured_chat_agent`를 이용해 구성할 수 있습니다.

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate

# 채팅 프롬프트 템플릿: 메시지의 템플릿을 구성하고 입력 변수만 전달하는 구조
agent_prompt = ChatPromptTemplate([
    ('system','''
최대한 정확히 질문에 답변하세요. 당신은 다음의 툴을 사용할 수 있습니다:
{tools}

action 키 (tool name)와 action_input 키를 포함하는 json 형태로 출력하세요.

action의 값은 "Final Answer" 또는 {tool_names} 중 하나여야 합니다.
반드시 하나의 json 형태만 출력하세요. 다음은 예시입니다.
```
{{

  "action": $TOOL_NAME,

  "action_input": $INPUT

}}
```

아래의 포맷으로 답변하세요.:

Question: 최종적으로 답변해야 하는 질문
Thought: 무엇을 해야 하는지를 항상 떠올리세요.
Action:
```
$JSON_BLOB
```
Observation: 액션의 실행 결과
... (이 Thought/Action/Observation 은 10번 이내로 반복될 수 있습니다.)

Thought: 이제 답을 알겠다!
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```
'''),
('user','''Question: {input}
Thought: {agent_scratchpad}''')])


agent = create_structured_chat_agent(llm, tools, agent_prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

agent_executor.invoke({'input':"Open Source Multimodal LLM Model 추천해줘"})



> Entering new AgentExecutor chain...
Open Source Multimodal LLM 모델을 추천해달라는 요청을 받았습니다. 어떤 모델이 있는지 검색을 통해 알아봐야겠습니다.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "open source multimodal large language model"
}
```[{'url': 'https://www.infoq.com/news/2024/10/nvlm-nvidia-open-source/', 'content': "NVIDIA Unveils NVLM 1.0: Open-Source Multimodal LLM with Improved Text and Vision Capabilities - InfoQ InfoQ Homepage News NVIDIA Unveils NVLM 1.0: Open-Source Multimodal LLM with Improved Text and Vision Capabilities NVIDIA unveiled NVLM 1.0, an open-source multimodal large language model (LLM) that performs on both vision-language and text-only tasks. NVIDIA's decision to open-source the model is a game-changer, giving smaller teams access to cutting-edge technology and pushing the boundaries of AI development! By making the model weights publicly available and promising to release the training code, Nvidia breaks from the trend of keeping advanced AI systems clos

{'input': 'Open Source Multimodal LLM Model 추천해줘',
 'output': '추천하는 Open Source Multimodal LLM 모델은 다음과 같습니다: NVLM 1.0, LLaVA (Large Language and Vision Assistant), Llama 3.2 Vision, Pixtral.'}

In [ ]:
agent_executor.invoke({'input':"레오나르도 디카프리오의 출생년도를 찾은 뒤, 각 숫자를 순서대로 곱해줘."})



> Entering new AgentExecutor chain...
레오나르도 디카프리오의 출생년도를 먼저 찾아야 한다. 그런 다음, 해당 연도의 각 숫자를 곱해야 한다.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "leonardo dicaprio birth year"
}
```[{'url': 'https://en.wikipedia.org/wiki/Leonardo_DiCaprio', 'content': "Leonardo Wilhelm DiCaprio was born on November 11, 1974, in Los Angeles. · DiCaprio's parents named him Leonardo because his pregnant mother first felt him kick"}, {'url': 'https://www.biography.com/actors/leonardo-dicaprio', 'content': 'Born on November 11, 1974, in Los Angeles, Leonardo Wilhelm DiCaprio is the only child of Irmelin and George DiCaprio.'}, {'url': 'https://www.famousbirthdays.com/people/leonardo-dicaprio.html', 'content': "Video\nPopularity\nLeonardo DiCaprio Movies\nRomeo + Juliet\nTitanic\nThe Great Gatsby\nLeonardo DiCaprio Fans Also Viewed\nKate Winslet\nMovie Actor\nBar Refaeli\nMovie Actor\nKirk Cameron\nMovie Actor\nClaire Danes\nMovie Actor\nMore November 11 Birthdays\nRichboy. Leonard

{'input': '레오나르도 디카프리오의 출생년도를 찾은 뒤, 각 숫자를 순서대로 곱해줘.',
 'output': '레오나르도 디카프리오의 출생년도는 1974년이며, 각 숫자를 곱한 결과는 252입니다.'}

랭체인의 에이전트 구조는 매우 간결하지만, 컨트롤하기가 매우 어려운데요.    
실제로 우리가 개발하는 구조에서는, 모든 단계에서 LLM이 컨텍스트를 저장하여 판단을 내릴 필요는 없습니다.   
또한, 모든 과정이 자연어로 구성되기 때문에 중간에 파싱 오류가 발생하는 경우에 동작이 멈추기도 합니다.   


특정 작업을 반복 실행하거나, 정해진 순서에 따라 실행해야 하는 상황이라면 어떻게 해야 할까요?    
랭체인으로 이와 같은 기능을 구현하는 것도 가능하지만, 다소 복잡한데요.   


랭그래프를 이용하면 구체적인 Workflow를 바탕으로 Agent 형태의 어플리케이션을 효과적으로 만들 수 있습니다.   
